In [26]:
# System imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np

pn.extension()

%matplotlib inline

In [25]:
# Local imports
import sys
sys.path.append("../lib2")

from Constants import Constants
from ProcessingTools import DateProcessingTools

In [22]:
atlas2          = pd.read_pickle(constants.PREPROCESSED_ATLAS_2_FILE_PATH)

NameError: name 'constants' is not defined

In [6]:
mapbox_token = os.getenv("MAPBOX_API_KEY")
load_dotenv()

True

In [17]:
# Import the CSVs to Pandas DataFrames
file_path = Path("../Resources/02_Anonymized/ATLAS with Address.csv")
sfo_data = pd.read_csv(file_path, index_col="Customers")
sfo_data.columns = ["Customers", "Lat", "Long"]

ValueError: Length mismatch: Expected axis has 10 elements, new values have 3 elements

In [18]:
sfo_data.head()

,Number of Users,Invoice Date,Invoice #,Invoice Amount,Subscription,Account Code,Dates of service,Address,Lat,Long
Customers,,,,,,,,,,
University 1,NaN,3/20/2015,ATLAS 315,"$72,000.00",1 Year,4700-0-00-00000-18-0000,1 Year Subscription 3/18/15 to 6/30/16,"1201 N 3rd St #6, Baton Rouge, LA 70802",30.460190,-91.188530
University 2,250.0,5/28/2015,AJ501,"$3,500.00",1 Year,4700-0-00-00000-16-0000,1 Year Subscription 6/1/15 to 6/30/16,"1 Kellogg Cir, Emporia, KS 66801",38.392609,-96.181396
University 3,100.0,6/23/2015,AJ502,"$3,500.00",1 Year,4700-0-00-00000-17-0000,1 Year Subscription 6/10/15 to 6/30/16,"150 University Blvd, Morehead, KY 40351",38.184921,-83.434441
University 4,NaN,6/26/2015,AJ503,"$6,500.00",1 Year,4700-0-00-00000-32-0000,1 Year Subscription 6/1/15 to 6/30/16,"44 Pierrepont Ave, Potsdam, NY 13676",44.663780,-74.978409
University 5,NaN,10/7/2015,AJ504,$750.00,1 Year,4700-0-00-00000-20-0000,1 Year Subscription 10/05/15 to 9/30/16,"47645 College Dr, St Marys City, MD 20686",38.190601,-76.426300


In [19]:
# Calculate the mean values for each neighborhood
mean_data = sfo_data.groupby(["Customers"]).mean()
mean_data.reset_index(inplace=True)

In [21]:
def neighborhood_map():
    """Neighborhood Map"""
    mean_data = sfo_data.groupby(["Customers"]).mean()
    mean_data.reset_index(inplace=True)
    px.set_mapbox_access_token(mapbox_token)
    neighborhood_map_plot = px.scatter_mapbox(
            mean_data,
            lat="Lat",
            lon="Long",
            size="Subscription",
            color="Customer"
        )
    return neighborhood_map_plot
neighborhood_map()

ValueError: Value of 'size' is not the name of a column in 'data_frame'. Expected one of ['Customers', 'Number of Users', 'Lat', 'Long'] but received: Subscription

In [ ]:
# System imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np
import dateparser
import panel as pn

pn.extension()

%matplotlib inline

In [ ]:
# Local imports
import sys
sys.path.append("../lib2")

from Constants import Constants
from ProcessingTools import DateProcessingTools

In [ ]:
# Construct the tools
debug_level = 0
constants = Constants()
tool_data_processing = DateProcessingTools(debug_level)


In [ ]:
# Import all preprocessed data
atlas           = pd.read_pickle(constants.PREPROCESSED_ATLAS_FILE_PATH)
forecast        = pd.read_pickle(constants.PREPROCESSED_FORECAST_DATA_FILE_PATH)
revenue2020     = pd.read_pickle(constants.PREPROCESSED_REVENUE2020_FILE_PATH)
revenue2020A    = pd.read_pickle(constants.PREPROCESSED_REVENUE2020A_FILE_PATH)
atlas2          = pd.read_pickle(constants.PREPROCESSED_ATLAS_2_FILE_PATH)
